In [209]:
import pandas
work = open('pg7166.txt').read().decode('utf-8','ignore')
work = work.replace('\r',' ').replace('\n',' ').lower()
puncts = ["'",'"',',','.',';',':','/','\\','(',')','?','!','@','#','%','^','&','*']
for item in puncts:
    work = work.replace(item,'')
    
work = work.replace('--',' ')
work = work.replace('-',' ')
work = work.replace('_',' ')
work = work.replace("  "," ")
work = work.replace('harsher', ' ')
st = open('english.txt').read().split('\n')
for item in st:
        work = work.replace(' '+item+' ',' ')
work = work.replace('_',' ')
words = work.split()



uniqWords = set(words)
print len(words),len(uniqWords)

30986 6633


In [229]:
wordFreq = dict()
biwordFreq = dict()
for item in words:
    try:
        wordFreq[item] += 1
    except:
        wordFreq[item] = 1
        
for i,item in enumerate(words):
    try:
        biwordFreq[item+' '+words[i+1]] += 1
    except:
        try:
            biwordFreq[item+' '+words[i+1]] = 1
        except:
            pass


In [263]:
bipd = pandas.DataFrame(biwordFreq.items(),columns=['biword','counta'])
bipd['wordie'] = bipd['biword'].str.split()
bipd['word1'] = bipd['wordie'].str[0]
bipd['word2'] = bipd['wordie'].str[1]
del bipd['biword']
del bipd['wordie']
bipd.to_csv('bigrams.csv',encoding='utf-8')

In [212]:
pd = pandas.DataFrame(wordFreq.items(),columns=['word','count'])
pd['len'] = pd['word'].str.len()
#pd.to_csv('uniqWords.csv',encoding='utf-8')
pd.sort_values(by=['count'],ascending=False)

word  count  len
3508        said    250    4
6499        will    231    4
5043        must    220    4
2026      sandip    220    6
566          one    209    3
2108         can    153    3
401         time    149    4
5653        come    146    4
269         life    135    4
4896        like    134    4
4694         day    129    3
251      country    127    7
242        never    126    5
2161         now    122    3
2283     husband    120    7
727        world    120    5
1848        away    118    4
5755      amulya    116    6
1077         let    115    3
3299        even    112    4
185        shall    111    5
196         came    111    4
3345        room    111    4
3797         man    110    3
1203          us    109    2
6602        made    106    4
6552        know    106    4
3934         get    102    3
1056         see     99    3
3792         may     98    3
...          ...    ...  ...
2868      oclock      1    6
2867        nose      1    4
2864     compact      1    7
2862    idolized      1    8
2860  harbouring      1   10
2902     singing      1    7
2858    ignorant      1    8
2857       bribe      1    5
2854    clenched      1    8
2853  humiliated      1   10
2851  ruthlessly      1   10
2848   pretended      1    9
2873    prudence      1    8
2876       stiff      1    5
2878       amras      1    5
2879      dilute      1    6
2881   punishing      1    9
2882      pepper      1    6
2884        jump      1    4
2885      picked      1    6
2886      ardent      1    6
2887       plays      1    5
2888     bluntly      1    7
2889        cell      1    4
2891      rosary      1    6
2892     likened      1    7
2893     noisily      1    7
2894       chant      1    5
2901   anomalies      1    9
3316    piercing      1    8

[6633 rows x 3 columns]

In [166]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]
xa = 'ceating'
print levenshtein(xa,'heating')
print levenshtein(xa,'eating')

1
1


In [195]:
for item in uniqWords:
            lev = levenshtein(item,'hareesh')
            jl = _jaro_winkler(item,'hareesh')*1.0
            if lev <=2:
                print item,lev,jl
            if jl > 0.89:
                print item,jl

harsher 0.894444444444
harish 2 0.894444444444
harish 0.894444444444
hardest 2 0.866666666667
harness 2 0.866666666667


In [182]:
from collections import defaultdict
_range = range
def _jaro_winkler(ying, yang, long_tolerance=False, winklerize=True):
    ying_len = len(ying)
    yang_len = len(yang)

    if not ying_len or not yang_len:
        return 0

    min_len = max(ying_len, yang_len)
    search_range = (min_len // 2) - 1
    if search_range < 0:
        search_range = 0

    ying_flags = [False]*ying_len
    yang_flags = [False]*yang_len

    # looking only within search range, count & flag matched pairs
    common_chars = 0
    for i, ying_ch in enumerate(ying):
        low = i - search_range if i > search_range else 0
        hi = i + search_range if i + search_range < yang_len else yang_len - 1
        for j in _range(low, hi+1):
            if not yang_flags[j] and yang[j] == ying_ch:
                ying_flags[i] = yang_flags[j] = True
                common_chars += 1
                break

    # short circuit if no characters match
    if not common_chars:
        return 0

    # count transpositions
    k = trans_count = 0
    for i, ying_f in enumerate(ying_flags):
        if ying_f:
            for j in _range(k, yang_len):
                if yang_flags[j]:
                    k = j + 1
                    break
            if ying[i] != yang[j]:
                trans_count += 1
    trans_count /= 2

    # adjust for similarities in nonmatched characters
    common_chars = float(common_chars)
    weight = ((common_chars/ying_len + common_chars/yang_len +
              (common_chars-trans_count) / common_chars)) / 3

    # winkler modification: continue to boost if strings are similar
    if winklerize and weight > 0.7 and ying_len > 3 and yang_len > 3:
        # adjust for up to first 4 chars in common
        j = min(min_len, 4)
        i = 0
        while i < j and ying[i] == yang[i] and ying[i]:
            i += 1
        if i:
            weight += i * 0.1 * (1.0 - weight)

        # optionally adjust for long strings
        # after agreeing beginning chars, at least two or more must agree and
        # agreed characters must be > half of remaining characters
        if (long_tolerance and min_len > 4 and common_chars > i+1 and
                2 * common_chars >= min_len + i):
            weight += ((1.0 - weight) * (float(common_chars-i-1) / float(ying_len+yang_len-i*2+2)))

    return weight

In [186]:
print 
print jellyfish.jaro_winkler(u'harish',u'hareesh')

0.894444444444
0.894444444444


In [202]:
pd.loc[pd['word'] == 'harsher']

word  count  len
1489  harsher      1    7